# Device-specific field calibration

The calibration of a cosmic-ray neutron probe (CRNP) is an essential step to ensure accurate soil moisture measurements. The CRNP operates by counting fast neutrons produced from cosmic rays, which are predominantly moderated by water molecules in the soil. The parameter $N_0$ is typically considered a device-specific constant that represents the neutron count rate in the absence of soil moisture conditions. 

$\theta(N) =\frac{a_0}{(\frac{N}{N_0}) - a_1} - a_2 $ (Desilets et al., 2010).

For the calibration of the stationary detector, a total of 14 undisturbed soil cores were collected at radial distances of 5, 50, and 100 m from the detector. In this example each soil sample was split into four depth segments: 0-5 cm, 5-10 cm, 10-25 cm, and 25-40 cm. Soil samples were processed and soil moisture was determined using the thermo-gravimetric method.

<img src="../../../img/layout.png" style="max-width:500px">

Figure 1. Horizontal layout and vertical layout used in this particular example calibration, it can be customized by the user depending on their needs.

[Download the following template](https://github.com/soilwater/crnpy/blob/main/docs/examples/calibration/template.xlsx)  for collecting your own calibration soil samples:

<img src="../../../img/template.png" style="max-width:500px">


In [26]:
# Importing required libraries
import crnpy

import numpy as np
import pandas as pd
from scipy.optimize import root


## Read calibration field survey data

For each sample it is required to know the bulk density ($\rho_\beta$) and the volumetric water content ($\theta_v$). See the details of the calculation used in the [filled example](https://github.com/soilwater/crnpy/blob/main/docs/examples/calibration/soil_data.csv). 


In [27]:
# Load the soil samples data and the CRNP dataset using pandas
url_soil_samples = "https://raw.githubusercontent.com/soilwater/crnpy/main/docs/examples/calibration/soil_data.csv"
df_soil = pd.read_csv(url_soil_samples)
df_soil.head(3)


,field,date,core_number,distance_from_station,latitude,longitude,top_depth,bottom_depth,core_diameter,wet_mass_with_bag,...,can_number,mass_empty_can,wet_mass_with_can,dry_mass_with_can,mass_water,theta_g,volume,bulk_density,theta_v,Observation
0,Flickner,22-Oct,1,5,N38.23459,W97.57101,0,5,30.49,45.31,...,1,52.10,92.03,85.31,6.72,0.202,36.514864,0.909,0.184403,NaN
1,Flickner,22-Oct,1,5,N38.23459,W97.57101,5,10,30.49,69.53,...,2,51.85,115.97,103.85,12.12,0.233,36.514864,1.424,0.332585,NaN
2,Flickner,22-Oct,1,5,N38.23459,W97.57101,10,25,30.49,214.90,...,3,51.56,260.97,219.77,41.20,0.245,109.544592,1.536,0.376856,NaN


In [28]:
# Define start and end of field survey calibration
calibration_start = pd.to_datetime("2021-10-22 08:00")
calibration_end = pd.to_datetime("2021-10-22 16:00")


## Read data from CRNP

In [29]:
# Load the station data
url_station_data = "https://raw.githubusercontent.com/soilwater/crnpy/main/docs/examples/calibration/station_data.csv"
df_station = pd.read_csv(url_station_data, skiprows=[0,2,3])

#  Parse dates (you can also use the option `parse_dates=['TIMESTAMP]` in pd.read_csv()
df_station['TIMESTAMP'] = pd.to_datetime(df_station['TIMESTAMP'], format='%Y-%m-%d %H:%M:%S')

df_station.head(3)


,TIMESTAMP,RECORD,station,farm,field,latitude,longitude,altitude,battery_voltage_Min,PTemp_Avg,...,wind_direction_Avg,wind_speed_gust_Max,air_temperature_Avg,vapor_pressure_Avg,barometric_pressure_Avg,relative_humidity_Avg,humidity_sensor_temperature_Avg,tilt_north_south_Avg,tilt_west_east_Avg,NDVI_Avg
0,2021-09-22 12:00:00,0,KS003,Flickner,Rainfed South,38.23461,-97.57095,455,13.52,29.04,...,352.3,4.20,22.30,9.20,973,41.30,26.4,-1.000,1.000,0.311
1,2021-09-22 13:00:00,1,KS003,Flickner,Rainfed South,38.23461,-97.57095,455,13.53,29.98,...,291.6,9.02,22.90,9.08,972,32.63,26.8,-0.975,0.950,0.308
2,2021-09-22 14:00:00,2,KS003,Flickner,Rainfed South,38.23461,-97.57095,455,13.53,30.43,...,292.8,5.54,23.38,8.68,971,30.25,27.2,-0.775,0.625,0.31


In [30]:
# Select station data only until the calibration date

# Define date in which the probe was deployed in the field (i.e., first record)
deployment_date = df_station['TIMESTAMP'].iloc[0]

# Filter station data from the first record to the end of the field survey calibration
# This is important since we are considering the incoming flux on the first day as the reference value
idx_period = (df_station['TIMESTAMP'] >= deployment_date) & (df_station['TIMESTAMP'] <= calibration_end)
df_station = df_station[idx_period]
df_station.head(3)


,TIMESTAMP,RECORD,station,farm,field,latitude,longitude,altitude,battery_voltage_Min,PTemp_Avg,...,wind_direction_Avg,wind_speed_gust_Max,air_temperature_Avg,vapor_pressure_Avg,barometric_pressure_Avg,relative_humidity_Avg,humidity_sensor_temperature_Avg,tilt_north_south_Avg,tilt_west_east_Avg,NDVI_Avg
0,2021-09-22 12:00:00,0,KS003,Flickner,Rainfed South,38.23461,-97.57095,455,13.52,29.04,...,352.3,4.20,22.30,9.20,973,41.30,26.4,-1.000,1.000,0.311
1,2021-09-22 13:00:00,1,KS003,Flickner,Rainfed South,38.23461,-97.57095,455,13.53,29.98,...,291.6,9.02,22.90,9.08,972,32.63,26.8,-0.975,0.950,0.308
2,2021-09-22 14:00:00,2,KS003,Flickner,Rainfed South,38.23461,-97.57095,455,13.53,30.43,...,292.8,5.54,23.38,8.68,971,30.25,27.2,-0.775,0.625,0.31


>This step is useful to trim large timeseries. For instance, the station data in our case extends until `2022-07-11 09:45:00`, but the field calibration was conducted in `2021-10-22 16:00`. Since all the station observation after the date of the field calibration are not relevent, we decided to only work with the data that we need from `2021-09-22 12:00:00` until `2021-10-22 16:00`. This could help getting data of incoming neutron flux from a single reference neutron monitor. So, if you are running this code shortly after the calibration field survey, then there is no need to filter station data.


## Correct neutron counts
                                                 

In [31]:
# Compute total neutron counts by adding the counts from both probe detectors
df_station['total_raw_counts'] = crnpy.compute_total_raw_counts(df_station[['counts_1_Tot','counts_2_Tot']],
                                                    nan_strategy='average')


In [32]:
# Atmospheric corrections

# Fill NaN values in atmospheric data
df_station[['barometric_pressure_Avg','relative_humidity_Avg', 'air_temperature_Avg']] = crnpy.fill_missing_atm(df_station[['barometric_pressure_Avg','relative_humidity_Avg', 'air_temperature_Avg']])

# Calculate absolute humidity
df_station['abs_humidity'] = crnpy.estimate_abs_humidity(df_station['relative_humidity_Avg'], df_station['air_temperature_Avg'])

# Compute correction factor for atmospheric pressure
# Reference atmospheric pressure for the location is 976 Pa
# Using an atmospheric attentuation coefficient of 130 g/cm2
df_station['fp'] = crnpy.pressure_correction(pressure=df_station['barometric_pressure_Avg'],
                                             Pref=976, L=130)

# Compute correction factor for air humidity
df_station['fw'] = crnpy.humidity_correction(abs_humidity=df_station['abs_humidity'],
                                             temp=df_station['air_temperature_Avg'],
                                             Aref=0)


In [33]:
# Find the cutoff rigidity for the location
cutoff_rigidity = crnpy.cutoff_rigidity(39.1, -96.6)

# Filtering the time window from experiment setup to the end of the calibration
crnpy.find_neutron_monitor(cutoff_rigidity,
                           start_date=df_station['TIMESTAMP'].iloc[0],
                           end_date=df_station['TIMESTAMP'].iloc[-1])



Select a station with an altitude similar to that of your location. For more information go to: 'https://www.nmdb.eu/nest/help.php#helpstations

Your cutoff rigidity is 2.87 GV
     STID     NAME     R  Altitude_m  Period available
13   DRBS  Dourbes  3.18         225              True
40   NEWK   Newark  2.40          50              True
28  KIEL2   KielRT  2.36          54              True


,STID,NAME,R,Altitude_m,Period available
13,DRBS,Dourbes,3.18,225,True
40,NEWK,Newark,2.40,50,True
28,KIEL2,KielRT,2.36,54,True


In [34]:
# Incoming neutron flux correction

# Download data for the reference neutron monitor and add to the DataFrame
incoming_neutrons = crnpy.get_incoming_neutron_flux(deployment_date,
                                                    calibration_end,
                                                    station="DRBS",
                                                    utc_offset=-5)

# Interpolate incoming neutron flux to match the timestamps in our station data
df_station['incoming_flux'] = crnpy.interpolate_incoming_flux(incoming_neutrons,
                                                              timestamps=df_station['TIMESTAMP'])

# Compute correction factor for incoming neutron flux
df_station['fi'] = crnpy.incoming_flux_correction(incoming_neutrons=df_station['incoming_flux'],
                                                  incoming_Ref=incoming_neutrons.iloc[0])



In [35]:
# Apply correction factors
df_station['total_corrected_neutrons'] = df_station['total_raw_counts'] * df_station['fw'] / (df_station['fp'] * df_station['fi'])


### Determine field-average soil moisture and bulk density

Using the function [`nrad_weight()`](../../../reference/#crnpy.crnpy.nrad_weight) the weights corresponding to each soil sample will be computed considering air-humidity, sample depth, distance from station and bulk density.


In [36]:
# Compute the weights of each sample for the field average
nrad_weights = crnpy.nrad_weight(df_station['abs_humidity'].mean(), df_soil['theta_v'], df_soil['distance_from_station'], (df_soil['bottom_depth']+df_soil['top_depth'])/2, rhob=df_soil['bulk_density'].mean())

# Apply distance weights to volumetric water content and bulk density
field_theta_v = np.sum(df_soil['theta_v']*nrad_weights)
field_bulk_density = np.sum(df_soil['bulk_density']*nrad_weights)


In [37]:
# Determine the mean corrected counts during the calibration survey
idx_cal_period = (df_station['TIMESTAMP'] >= calibration_start) & (df_station['TIMESTAMP'] <= calibration_end)
mean_cal_counts = df_station.loc[idx_cal_period, 'total_corrected_neutrons'].mean()

print(f"Mean volumetric Water content during calibration survey: {round(field_theta_v,3)}")
print(f"Mean corrected counts during calibration: {round(mean_cal_counts)} counts")


Mean volumetric Water content during calibration survey: 0.263
Mean corrected counts during calibration: 1546 counts


## Solving for $N_0$

Previous steps estimated the a field volumetric water content of `0.263` and an average neutron count of `1537`. Using [`scipy.optimize.root()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.root.html) $N_0$ is estimated given the observed value of $\theta_v$ and neutron counts.

In [38]:
# Define the function for which we want to find the roots
VWC_func = lambda N0 : crnpy.counts_to_vwc(mean_cal_counts, N0, bulk_density=field_bulk_density, Wlat=0.03, Wsoc=0.01) - field_theta_v

# Make an initial guess for N0
N0_initial_guess = 1000

# Find the root
sol = int(root(VWC_func, N0_initial_guess).x)

# Print the solution
print(f"The solved value for N0 is: {sol}")


The solved value for N0 is: 2650


## References: 
Desilets, D., Zreda, M., & Ferré, T. P. (2010). Nature's neutron probe: Land surface hydrology at an elusive scale with cosmic rays. Water Resources Research, 46(11).

Dong, J., & Ochsner, T. E. (2018). Soil texture often exerts a stronger influence than precipitation on mesoscale soil moisture patterns. Water Resources Research, 54(3), 2199-2211.

Patrignani, A., Ochsner, T. E., Montag, B., & Bellinger, S. (2021). A novel lithium foil cosmic-ray neutron detector for measuring field-scale soil moisture. Frontiers in Water, 3, 673185.